___
# Ciência dos Dados - PROJETO 2

___
## Jhonata Ferreira

## Matheus Augusto 

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [1]:
import pandas as pd
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\mathe\Desktop\Ano Da DP\Ciencia dos Dados\Projeto 022


In [3]:
dados = pd.read_excel('spamhamV2.xlsx',sep=',')
dados.head()

,Email,Class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [4]:
treinamento=dados.sample(frac=0.75)
teste=dados.drop(treinamento.index)

In [5]:
treinamento.head()

,Email,Class
3854,Free msg: Single? Find a partner in your area!...,spam
2466,Is there coming friday is leave for pongal?do ...,ham
5554,Well keep in mind I've only got enough gas for...,ham
4602,And I don't plan on staying the night but I pr...,ham
2389,First has she gained more than &lt;#&gt; kg s...,ham


In [6]:
teste.head()

,Email,Class
1,Ok lar... Joking wif u oni...,ham
7,As per your request 'Melle Melle (Oru Minnamin...,ham
8,WINNER!! As a valued network customer you have...,spam
10,I'm gonna be home soon and i don't want to tal...,ham
13,I've been searching for the right words to tha...,ham


In [7]:
Ham_t = treinamento.loc[treinamento.Class == 'ham']
Spam_t = treinamento.loc[treinamento.Class == 'spam']

In [56]:
#Criando listas com palavras

Ham_palavras = []
for email in Ham_t.Email:
    palavras = email.split()
    for palavra in palavras:
        Ham_palavras.append(palavra)
        
Spam_palavras = []
for email in Spam_t.Email:
    palavras = email.split()
    for palavra in palavras:
        Ham_palavras.append(palavra)

Treinamento_palavras = []
for email in treinamento.Email:
    palavras = email.split()
    for palavra in palavras:
        Treinamento_palavras.append(palavra)

In [57]:
def clean(lista):
    i=0
    while i < len (lista):
        lista[i]=lista[i].replace("_"," ")
        lista[i]=lista[i].replace("'"," ")
        lista[i]=lista[i].replace("?"," ")
        lista[i]=lista[i].replace("$"," ")
        lista[i]=lista[i].replace("*"," ")
        lista[i]=lista[i].replace("%"," ")
        lista[i]=lista[i].replace("-"," ")
        lista[i]=lista[i].replace("<"," ")
        lista[i]=lista[i].replace(">"," ")
        lista[i]=lista[i].replace("'"," ")
        lista[i]=lista[i].replace('"'," ")
        lista[i]=lista[i].replace("!"," ")
        lista[i]=lista[i].replace("."," ")
        lista[i]=lista[i].replace(","," ")
        lista[i]=lista[i].replace("|"," ")
        lista[i]=lista[i].replace(":"," ")
        lista[i]=lista[i].replace("&"," ")
        lista[i]=lista[i].replace("("," ")
        lista[i]=lista[i].replace(")"," ")
        lista[i]=lista[i].replace("+"," ")
        lista[i]=lista[i].replace("#"," ")
        lista[i]=lista[i].replace("^"," ")
        lista[i]=lista[i].replace("÷"," ")
        lista[i]=lista[i].replace("and"," ")
        lista[i]=lista[i].replace("is"," ")
        i+=1
    lista=list(filter(None, lista))

    return lista

In [59]:
ham_clean=clean(Ham_palavras)
spam_clean=clean(Spam_palavras)
treinamento_clean=(Treinamento_palavras)

___
## 3. Classificador Naive-Bayes

___
## 4. Curva ROC

___
## 5. Qualidade do Classificador alterando a base de treinamento